In [ ]:
# Fine tuning gpt2_medium model and use own data like company profile
#
# See also medium.com blog
# "GPT-2 Fine-Tuning Guide: Building a Chatbot for Your Company Profile"
# https://medium.com/@datatec.studio

In [ ]:
# Install python packages
# !pip install -r requirements.txt

In [1]:
import os
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from torch.optim import Adam
from torch.utils.data import DataLoader
from tqdm import tqdm
import time
import numpy as np
import sys

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Define environment variable, path of data, model name and device
# os.environ["HF_HOME"] = "/content/huggingface"  # Replace with your desired directory
# print("Please replace it with your hf access token:")
# os.environ["HF_HOME_TOKEN"] = "Please_replace_it_with_your_hf_access_token"

result_dir = 'resources/'
data_file_name = 'Jeremy'
data_file_path = f'../data/prompt_response/{data_file_name}.txt'

model_name = "gpt2" # gpt2-medium
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(device)

cuda


In [3]:
# Write a python file to google driver
# Sample of json datasets
# You can also directly upload this code to your google driver
# The code write here in this way is for better understanding of whole project
# %%writefile chat_data.py

from torch.utils.data import Dataset
import json

class ChatData(Dataset):
    def __init__(self, path: str, tokenizer):
        with open(path, encoding="utf-8") as f:
            self.data = f.readlines()#json.load(open(path, "r"))

        self.X = []
        for pair in self.data:
            pair = eval(pair)          
            self.X.append(f"<start> {pair['prompt']} <response>: {pair['response']} <end>")
        
        total_samples = len(self.X)  # Calculate the total number of samples
        print("total_samples", total_samples)
        # define samples amount
#         self.X = self.X[:500]
        print("Check the preprocessing for self.X[0]:")
        print(self.X[0])

        self.X_encoded = tokenizer(self.X, return_tensors="pt", max_length=30, padding="max_length", truncation=True)
        self.input_ids = self.X_encoded['input_ids']
        self.attention_mask = self.X_encoded['attention_mask']

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.attention_mask[idx]


In [4]:
# Download model, save model and tokernize to harddisk
## prepare tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

print(f'Number of tokens in tokenizer before adding our specific tokens: {len(tokenizer.get_vocab())}')

tokenizer.add_special_tokens({"pad_token": "<pad>",
                              "bos_token": "<start>",
                              "eos_token": "<end>"})

tokenizer.add_tokens(["<response>:"])
with open(f'../data/prompt_response/groupchat.txt', encoding="utf-8") as f:
    data = f.readlines()#json.load(open(path, "r"))

for pair in data:
    pair = eval(pair)
    tokenizer.add_tokens(pair['prompt'].split() + pair['response'].split())
    
print(f'Number of tokens in tokenizer after adding our specific tokens: {len(tokenizer.get_vocab())}')

## prepare model
### Specify the desired embedding size (must be a multiple of 8)
desired_embedding_size = 50264  # Change this to the desired size
model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
### Resize the embedding layer to the desired size
model.resize_token_embeddings(len(tokenizer), desired_embedding_size)
model = model.to(device)

model.config.pad_token_id = tokenizer.eos_token_id + 1


## save tokenizer and model to harddisk
# tokenizer.save_pretrained(result_dir)
# model.save_pretrained(result_dir)

Number of tokens in tokenizer before adding our specific tokens: 50257
Number of tokens in tokenizer after adding our specific tokens: 52566


In [5]:
# ## load model and tokenizer from harddisk
# ### Load the GPT-2 tokenizer
# tokenizer = GPT2Tokenizer.from_pretrained(result_dir)

# ### Load the GPT-2 model from the local folder
# model = GPT2LMHeadModel.from_pretrained(result_dir)
# model.to(device)

In [6]:
# Define infer and train function
def infer(inp_raw):
    inp_appended = "<start> " + inp_raw + " <response>: "
    inp = tokenizer(inp_appended, return_tensors="pt")
    X = inp["input_ids"].to(device)  # Use .to(device) method to move the tensor to the specified device
    a = inp["attention_mask"].to(device)  # Use .to(device) method here as well

    output = model.generate(X, attention_mask=a, max_length=100, num_return_sequences=1)
    output = tokenizer.decode(output[0]).replace('<pad>', '')
    output = output[output.find('<response>: ')+len('<response>: '):].replace('<response>:', '')
    if '<end>' in output:
        output = output[:output.find('<end>')]
    output = ' '.join(output.split())
    
    prompt_response = {
        'prompt': inp_raw,
        'response': output
    }

    return prompt_response


def train(chatData, model, optim):
    
    batches = len(chatData)

    for i, (X, a) in tqdm(enumerate(chatData), total=len(chatData), desc="Training"):
        X = X.to(device)
        a = a.to(device)
        optim.zero_grad()
        loss = model(input_ids=X, attention_mask=a, labels=X).loss
        loss.backward()
        optim.step()

    print(infer("How's progress?"))
    print('========================================================')

In [7]:
# from chat_data import ChatData

#Load ChatData, train model and optimizer
chatData = ChatData(data_file_path, tokenizer)
chatData = DataLoader(chatData, batch_size=1) # batch_size=64

model.train()

optim = Adam(model.parameters(), lr=1e-5)

total_samples 164
Check the preprocessing for self.X[0]:
<start> What should our group name be 😎 Ester and I used transformers for PA1 <response>: lol anything works <end>


In [10]:
epochs = 100  # You can adjust the number of epochs as needed
for epoch in range(epochs):
    start = time.time()
    print(f"Epoch {epoch} started")
    train(chatData, model, optim)
    end = time.time()
    print(f"Epoch {epoch} finished in {np.round((end - start) / 60, 2)} minutes")

Epoch 0 started


Training: 100%|██████████| 164/164 [00:06<00:00, 25.57it/s]


{'prompt': "How's progress?", 'response': ''}
Epoch 0 finished in 0.12 minutes
Epoch 1 started


Training: 100%|██████████| 164/164 [00:06<00:00, 25.20it/s]


{'prompt': "How's progress?", 'response': ''}
Epoch 1 finished in 0.12 minutes
Epoch 2 started


...

{'prompt': "How's progress?", 'response': 'ok i updated the abstract, also added in the pictures of the photo focal of the photo ok i just pushed the bit but / i just added the pixel size of the photo to my related pixel is rollled'}
Epoch 97 finished in 0.12 minutes
Epoch 98 started


Training: 100%|██████████| 164/164 [00:06<00:00, 26.51it/s]


{'prompt': "How's progress?", 'response': 'ur actually amazing'}
Epoch 98 finished in 0.12 minutes
Epoch 99 started


Training: 100%|██████████| 164/164 [00:06<00:00, 27.30it/s]


{'prompt': "How's progress?", 'response': 'oh how many neurons are u using'}
Epoch 99 finished in 0.11 minutes


In [ ]:
inp = ""
while True:
    inp = input("Enter your input (press Enter when done): " + " " * 20)
    print(infer(inp))

In [11]:
# Save the model's state dictionary after training is complete
torch.save(model.state_dict(), f"models/{data_file_name}.pt")

# Simulate a Chat

In [12]:
ester = GPT2LMHeadModel.from_pretrained(model_name).to(device)
ester.resize_token_embeddings(len(tokenizer), desired_embedding_size)
ester.load_state_dict(torch.load("models/Ester.pt"))
ester.config.pad_token_id = tokenizer.eos_token_id + 1

winfrey = GPT2LMHeadModel.from_pretrained(model_name).to(device)
winfrey.resize_token_embeddings(len(tokenizer), desired_embedding_size)
winfrey.load_state_dict(torch.load("models/Winfrey.pt"))
winfrey.config.pad_token_id = tokenizer.eos_token_id + 1

jeremy = GPT2LMHeadModel.from_pretrained(model_name).to(device)
jeremy.resize_token_embeddings(len(tokenizer), desired_embedding_size)
jeremy.load_state_dict(torch.load("models/Jeremy.pt"))
jeremy.config.pad_token_id = tokenizer.eos_token_id + 1

jonathan = GPT2LMHeadModel.from_pretrained(model_name).to(device)
jonathan.resize_token_embeddings(len(tokenizer), desired_embedding_size)
jonathan.load_state_dict(torch.load("models/Jonathan.pt"))
jonathan.config.pad_token_id = tokenizer.eos_token_id + 1

samuel = GPT2LMHeadModel.from_pretrained(model_name).to(device)
samuel.resize_token_embeddings(len(tokenizer), desired_embedding_size)
samuel.load_state_dict(torch.load("models/Samuel.pt"))
samuel.config.pad_token_id = tokenizer.eos_token_id + 1

In [13]:
def group_infer(inp_raw, model):
    inp_appended = "<start> " + inp_raw + " <response>: "
    inp = tokenizer(inp_appended, return_tensors="pt")
    X = inp["input_ids"].to(device)  # Use .to(device) method to move the tensor to the specified device
    a = inp["attention_mask"].to(device)  # Use .to(device) method here as well

    output = model.generate(X, attention_mask=a, max_length=100, num_return_sequences=1)
    output = tokenizer.decode(output[0])
#     print(output)
    output = output[output.find('<response>: ')+len('<response>: '):].replace('<pad>', '')#.replace('<response>:', '')#.replace('<pad>', '')#.replace('<end>', '')
    if '<response>' in output:
        output = output[:output.find('<response>')]
    if '<end>' in output:
        output = output[:output.find('<end>')]
    output = ' '.join(output.split())
    
    prompt_response = {
        'prompt': inp_raw,
        'response': output
    }

    return prompt_response

In [117]:
speaker = 'Samuel'
output = group_infer("oh okay", eval(speaker.lower())) # How's it going?
with_name = f'{speaker}: ' + output['prompt']
convo = [with_name]
print(with_name)

people = ['Winfrey', 'Ester', 'Jeremy', 'Samuel', 'Jonathan']
for i in range(15):
    response = output['response']
    remaining_people = [person for person in people if person != speaker]
    speaker = np.random.choice(remaining_people)
#     print(speaker)
    output = group_infer(response, eval(speaker.lower())) # assuming models are named ester, winfrey, ...
    with_name = f'{speaker}: ' + output['response']
    convo += [with_name]
    print(with_name)

Samuel: oh okay
Jeremy: for different temperatures
Ester: can you all check on your end what your test accuracy is before training? your error or you can comment on the end by cloning your error or you guys will have a better test accuracy than baseline
Winfrey: Just set it at 10 torch.manual_seed(seed) for better.
Jonathan: I can also fill in the addition
Jeremy: check
Winfrey: ok check it in a bit
Jonathan: i can do a final read through if it messes up if theres diff


KeyboardInterrupt: 

In [ ]:
convo